### 进程创建

In [2]:
import multiprocessing as mp

#定义一个被进程调用的函数
def job(a,b):
    print('aaaa')

if __name__=='__main__':
    p1=mp.Process(target=job,args=(1,2))
    p1.start()
    p1.join()

aaaa


### 存储进程输出Queue

Queue的功能是将每个核或线程的运算结果放在队里中，等到每个线程或核运算完毕后再从队列中取出结果，继续加载运算。原因很简单，**多进程或多线程调用的函数不能有返回值**，所以使用Queue存储多个线程或进程运算的结果。

In [4]:
import multiprocessing as mp

#定义一个没有返回值得调用函数
def job(q,res):
    q.put(res)


if __name__=='__main__':
    q=mp.Queue()
    p1=mp.Process(target=job,args=(q,1))
    p2=mp.Process(target=job,args=(q,3))
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    res1=q.get()
    res2=q.get()
    print(res1)
    print(res2)

3
1


### 进程池Pool

进程池就是我们将所有运行的东西，放到池子里，python会自行解决多进程的问题。

1. pool默认调动是CPU的核数,Pool调用的函数有返回值，而Process调用的函数则没有返回值
2. 对Pool对象调用join()方法会等待所有子进程执行完毕，调用join()之前必须先调用close(),调用cloes()之后就不能继续添加新的Process
3. map(func,seq)放入迭代参数，返回多个结果
4. apply_async(func,parm)只能执行一次

In [2]:
import multiprocessing as mp

def job(x):
    return x*x

if __name__=='__main__':
    pool=mp.Pool()
    res=pool.map(job,range(10))
    print(res)

TypeError: job() missing 1 required positional argument: 'x'

In [6]:
from multiprocessing import Pool
import os,time,random

def long_time_test(name):
    print('Run task %s (%s)...'%(name,os.getpid()))
    start=time.time()
    time.sleep(random.random()*3)
    end=time.time()
    print('Task %s runs %0.2f seconds.'%(name,(end-start)))

if __name__=='__main__':
    print('Parent process %s.'%os.getpid())
    p=Pool(4)
    for i in range(5):
        p.apply_async(long_time_test,args=(i,))
    print('Waiting for all subprocess done...')
    p.close()
    p.join()
    print('All subprocess done.')

Parent process 1863.
Run task 1 (2098)...
Run task 0 (2097)...
Run task 3 (2100)...
Run task 2 (2099)...
Waiting for all subprocess done...
Task 3 runs 0.29 seconds.
Run task 4 (2100)...
Task 2 runs 1.75 seconds.
Task 0 runs 2.59 seconds.
Task 1 runs 2.72 seconds.
Task 4 runs 2.93 seconds.
All subprocess done.


### 共享内存

定义共享内存中的数据可以被不同进程读取和修改。

| Type code | C Type             | Python Type       | Minimum size in bytes |
| --------- | ------------------ | ----------------- | --------------------- |
| `'b'`     | signed char        | int               | 1                     |
| `'B'`     | unsigned char      | int               | 1                     |
| `'u'`     | Py_UNICODE         | Unicode character | 2                     |
| `'h'`     | signed short       | int               | 2                     |
| `'H'`     | unsigned short     | int               | 2                     |
| `'i'`     | signed int         | int               | 2                     |
| `'I'`     | unsigned int       | int               | 2                     |
| `'l'`     | signed long        | int               | 4                     |
| `'L'`     | unsigned long      | int               | 4                     |
| `'q'`     | signed long long   | int               | 8                     |
| `'Q'`     | unsigned long long | int               | 8                     |
| `'f'`     | float              | float             | 4                     |
| `'d'`     | double             | float             | 8                     |

In [7]:
import multiprocessing as mp

#通过使用Value将数据存储在一个共享的内存表中
#其中d和i参数用来设置数据类型
#d表示一个双精度浮点类型,i表示一个带符号的整型
value1=mp.Value('i',0)
value2=mp.Value('d',3.14)

#这里的Array只能是一维的，不能是多维的，且需要定义数据形式
array=mp.Array('i',[1,2,3,4])

### 进程锁Lock

In [14]:
#不加进程锁，两个进程争夺共享内存v
import multiprocessing as mp
import time,os

def job(v,num):
    for _ in range(5):
        time.sleep(num)
        v.value+=num   #v.value获取共享变量值
        print('curr process',os.getpid(),v.value)

def multicore():
    v=mp.Value('i',0)
    p1=mp.Process(target=job,args=(v,1))
    p2=mp.Process(target=job,args=(v,3))
    p1.start()
    p2.start()
    p1.join()
    p2.join()

if __name__=='__main__':
    multicore()

curr process 2749 1
curr process 2749 2
curr process 2750 5
curr process 2749 6
curr process 2749 7
curr process 2749 8
curr process 2750 11
curr process 2750 14
curr process 2750 17
curr process 2750 20


In [17]:
#使用进程锁，保证运行时一个进程对锁内内容的独占
#使用l.acquire()加锁,l.release()释放锁
import multiprocessing as mp
import time

def job(v,num,l):
    l.acquire()
    for _ in range(5):
        time.sleep(num)
        v.value+=num
        print(v.value)
    l.release()

def multicore():
    l=mp.Lock()
    v=mp.Value('i',0)
    p1=mp.Process(target=job,args=(v,3,l))
    p2=mp.Process(target=job,args=(v,1,l))
    p1.start()
    p2.start()
    p1.join()
    p2.join()

if __name__=='__main__':
    multicore()

3
6
9
12
15
16
17
18
19
20
